In [1]:
import cv2
import numpy as np
import os
import argparse
import face_recognition
import time

In [2]:
#cargar imagen de base de datos
image_julian = face_recognition.load_image_file("juliancontreras.jpg")
image_matias = face_recognition.load_image_file("matiascornejo.jpg")
julian_encodings = face_recognition.face_encodings(image_julian)[0]
matias_encodings = face_recognition.face_encodings(image_matias)[0]

In [3]:
#caras y nombres conocidos, en orden
known_faces = [julian_encodings, matias_encodings]
known_names = ["Julian Contreras", "Matias Cornejo"]

In [4]:
#inicialización de variables
face_locations = []
face_encodings = []
face_names = []
font = cv2.FONT_HERSHEY_DUPLEX
process_this_image = True
counter = 0

In [5]:
#captura de video
url = "http://192.168.31.15:8080/video"
cap = cv2.VideoCapture(url)

In [7]:
start_time = time.time()
while True:

    #leer imagen
    ret, frame = cap.read()
    
    #resize imagen, apurar procesamiento
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    #conversión BGR to RGB
    rgb_small_frame = small_frame[:, :, ::-1]

    #inicializacion nombres
    face_names = []

    #solo procesar si esta activo el video

    #procesar imagen, buscando caras
    face_locations = face_recognition.face_locations(rgb_small_frame, model="cnn")
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for face_encoding in face_encodings:

        #comparar caras
        match = face_recognition.compare_faces(np.array(known_faces), face_encoding, tolerance=0.35)
        name = "Unknown"

        for i in range(len(match)):

            #buscar similitud
            if match[i]:
                name = known_names[i]
                break

        face_names.append(name)
        

    #mostrar resultados
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        #resize imagen, para mostrar resultados correctamente
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        #dibujar rectangulo resultados  
        cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255), 2)
        
        #dibujar nombre
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0,0,255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255,255,255), 1)

    counter += 1
    #mostrar imagen
    cv2.imshow("Video", frame)

    #apretar "q" para salir
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

#datos bacan 
print("Elapsed Time: {}".format(time.time() - start_time))
print("Approx. FPS: {}".format(counter/(time.time() - start_time)))

# Cleaning up
cv2.destroyAllWindows()

Elapsed Time: 13.939326763153076
Approx. FPS: 59.32854678362763
